In [ ]:
# Autor: Abeil Coelho Júnior | Universidade Federal do Espírito Sando | PPGCI
# Data de criação: 01/06/2022
# Descrição: Exportar em lote todos os dados das coleções disponívies online pelo IBRAM.
# Versão: 1
# Data de modificação: 13/06/2022

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from time import sleep
from glob import glob
import pyarrow as pa
import requests
import math
import json

import collections
import itertools
import time
from pandas_profiling import ProfileReport
import csv
import html

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
inicio_geral = time.time()

In [2]:
url_museus = ["https://museudodiamante.acervos.museus.gov.br/acervo-museologico", 
              "https://museudoouro.acervos.museus.gov.br/acervo/", 
              "http://museuvictormeirelles.acervos.museus.gov.br/mvm-acervo/", 
              "https://museuregionalcasadosottoni.acervos.museus.gov.br/acervo-museologico/", 
              "https://museudasmissoes.acervos.museus.gov.br/acervo-museologico/", 
              "https://museusibramgoias.acervos.museus.gov.br/museu-das-bandeiras/", 
              "https://museusibramgoias.acervos.museus.gov.br/museu-casa-da-princesa/", 
              "https://museuvillalobos.acervos.museus.gov.br/fotografias/", 
              "https://museucasabenjaminconstant.acervos.museus.gov.br/acervo-museologico/", 
              "https://museudainconfidencia.acervos.museus.gov.br/acervo-museologico/", 
              "https://museuregionaldesaojoaodelrei.acervos.museus.gov.br/acervo_museologico/", 
              "https://museusibramgoias.acervos.museus.gov.br/museu-casa-da-boa-morte/", 
              "https://museucasadahera.acervos.museus.gov.br/acervo-museologico/", 
              "https://museucasahistoricadealcantara.acervos.museus.gov.br/objetos/", 
              "https://museudeartereligiosaetradicional.acervos.museus.gov.br/acervo-museologico/", 
              "https://museudaabolicao.acervos.museus.gov.br/acervo_museologico/", 
              "https://museuregionaldecaete.acervos.museus.gov.br/acervo/", 
              "https://museusolarmonjardim.acervos.museus.gov.br/acervo/",
              "https://mhn.acervos.museus.gov.br/moedas-de-ouro/",
              "https://museucasadahera.acervos.museus.gov.br/indumentaria/"]

# Coleções com exportação problemáticos
# "https://museudearqueologiadeitaipu.museus.gov.br/museu-itaipu/",
#               "https://mhn.acervos.museus.gov.br/reserva-tecnica/"

# Acervo arquivísitico
# "http://museusolarmonjardim.acervos.museus.gov.br/documentos-historicos/",

In [3]:
# Raspagem exportação em csv
def construcao_url(url_museu, id_colecao):
    # Pegar url base do museu
    url_clean = url_museu.split("/")[2]
    
    # Pegar nome do acervo
    nome = url_clean.split(".")[0]
    acervo = url_museu.split("/")[3]
    nome_unico = nome + " - " + acervo
    nome_unico = nome_unico.replace(" ", "_")
    
    # Construtir url de raspagem
    url_raspagem = "https://" + url_clean + "/wp-json/tainacan/v2/collection/" + str(id_colecao) + "/items/?perpage=96&order=ASC&orderby=date&exposer=csv&mapper=inbcm-ibram&paged="
    url_matadados = "http://" + url_clean + "/wp-json/tainacan/v2/collections/" + str(id_colecao)
    api_dados_tainacam = url_raspagem = "https://" + url_clean + "/wp-json/tainacan/v2/collection/" + str(id_colecao) + "/items/?perpage=1&order=ASC&orderby=date&paged=1"
    
    return url_raspagem, nome_unico, url_matadados, api_dados_tainacam

In [4]:
# url_raspagem, nome_unico, url_matadados, api_dados_tainacam = construcao_url("https://museudasmissoes.acervos.museus.gov.br/acervo-museologico/", "6409")
# #print(api_dados_tainacam)


In [38]:
# Teste

# headers = {
#   'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="100", "Opera GX";v="86"',
#   'sec-ch-ua-mobile': '?0',
#   'sec-ch-ua-platform': '"Windows"',
#   'Upgrade-Insecure-Requests': '1',
#   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 OPR/86.0.4363.64',
#   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
# }

# base_campos_controlados = pd.DataFrame()

# for url in url_museus:
#     print(url)
#     r = requests.get(url, timeout=5000, verify=False, headers=headers)
#     soup = BeautifulSoup(r.text, 'html.parser')
    
#     # Captar ID da coleção
#     #try:
#     col_id = soup.find(attrs={"id" : "tainacan-items-page"})
#     collection_id = col_id['collection-id']
    
#     nome_acervo = collection_id
#     url_raspagem, nome_unico, url_matadados, api_dados_tainacam = construcao_url(url, collection_id)

#     print(api_dados_tainacam)
#     vocabulario = requests.get(api_dados_tainacam)
#     binary = vocabulario.content
#     data = json.loads(binary)
#     register = data['items'][0]['metadata']

#     campos_controlados = []

#     # Listagem de campo inbcm que usam taxonomia
#     for key, value in register.items():
#         print(key)
#         if 'inbcm-ibram' in register[key]['mapping']:
#             try:
#                 try:
#                     if 'taxonomy' in register[key]['value'][0]:
#                         campo = register[key]['mapping']["inbcm-ibram"]
#                         campos_controlados.append(campo)
#                         print(register[key]['mapping']["inbcm-ibram"])
#                         print(register[key]['value'][0]["taxonomy"])
#                         del campo
#                 except:
#                     if 'taxonomy' in register[key]['value']:
#                         campo = register[key]['mapping']["inbcm-ibram"]
#                         campos_controlados.append(campo)
#                         print(register[key]['mapping']["inbcm-ibram"])
#                         print(register[key]['value']["taxonomy"])
#                         del campo
#             except:
#                 pass
            
    
#     colecoes_campos_controlados = pd.DataFrame(campos_controlados, columns=["Campos_Controlados"])
#     colecoes_campos_controlados["Colecao"] = nome_unico
#     frames = [colecoes_campos_controlados, base_campos_controlados]
#     base_campos_controlados = pd.concat(frames)
#     del colecoes_campos_controlados



#     matadata = requests.get(url_matadados)
#     matadata_json = matadata.json()
#     tamanho_acervo =  matadata_json['total_items']['publish']
#     tamanho_acervo = int(tamanho_acervo)

#     nome_acervo =  matadata_json['name']
#     print(url_museu, tamanho_acervo, nome_acervo)

https://museudodiamante.acervos.museus.gov.br/acervo-museologico
https://museudodiamante.acervos.museus.gov.br/wp-json/tainacan/v2/collection/11603/items/?perpage=1&order=ASC&orderby=date&paged=1
titulo-3
numero-de-registro-3
title-3
classe-3
classificacao
tnc_tax_42
local-de-producao
local-producao
tnc_tax_18430
data-de-producao
data-producao
tnc_tax_18432
acervo-3
procedencia-3
autor
autor
tnc_tax_18298
material-tecnica-3
material-tecnica
tnc_tax_11646
termos-de-indexacao-2
altura-cm-2
largura-cm
situacao-2
comprimento-cm
profundidadecm
diametro-cm
circunferenciacm
espessuracm
peso
description-3
data-da-foto-2
marcas-inscricoes-legendas-3
dados-historicos-2
referencias-bibliograficas-arquivisticas-3
condicoes-de-reproducao-2
midias-relacionadas


NameError: name 'url_museu' is not defined

In [4]:
# Raspagem exportação em HTML (caso raspagem dos CSV dê erro)
def construcao_url_html(url_museu, id_colecao):
    # Pegar url base do museu
    url_clean = url_museu.split("/")[2]
    
    # Pegar nome do acervo
    nome = url_clean.split(".")[0]
    acervo = url_museu.split("/")[3]
    nome_unico = nome + " - " + acervo
    nome_unico = nome_unico.replace(" ", "_")
    
    # Construtir url de raspagem
    url_raspagem = "http://" + url_clean + "/wp-json/tainacan/v2/collection/" + str(id_colecao) + "/items/?perpage=96&order=ASC&orderby=date&exposer=html&paged="
    
    return url_raspagem, nome_unico

In [6]:
total_registros = 0
acervos_que_falharam = []

headers = {
  'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="100", "Opera GX";v="86"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 OPR/86.0.4363.64',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',

}

base_campos_controlados = pd.DataFrame()
for url in url_museus:
    try:
        print("\n",url)

        session = HTMLSession()
        r = session.get(url, timeout=5000, verify=False, headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')

        # Captar ID da coleção

        col_id = soup.find(attrs={"id" : "tainacan-items-page"})
        collection_id = col_id['collection-id']

        nome_acervo = collection_id
        url_raspagem, nome_unico, url_matadados, api_dados_tainacam = construcao_url(url, collection_id)


        vocabulario = requests.get(api_dados_tainacam)
        binary = vocabulario.content
        data = json.loads(binary)
        register = data['items'][0]['metadata']

        campos_controlados = []
        # Listagem de campo inbcm que usam taxonomia
        for key, value in register.items():
            print(key)
            if 'inbcm-ibram' in register[key]['mapping']:
                try:
                    try:
                        if 'taxonomy' in register[key]['value'][0]:
                            campo = register[key]['mapping']["inbcm-ibram"]
                            campos_controlados.append(campo)
                            print(register[key]['mapping']["inbcm-ibram"])
                            print(register[key]['value'][0]["taxonomy"])
                            del campo
                    except:
                        if 'taxonomy' in register[key]['value']:
                            campo = register[key]['mapping']["inbcm-ibram"]
                            campos_controlados.append(campo)
                            print(register[key]['mapping']["inbcm-ibram"])
                            print(register[key]['value']["taxonomy"])
                            del campo
                except:
                    pass


        colecoes_campos_controlados = pd.DataFrame(campos_controlados, columns=["Campos_Controlados"])
        colecoes_campos_controlados["Colecao"] = nome_unico
        frames = [colecoes_campos_controlados, base_campos_controlados]
        base_campos_controlados = pd.concat(frames)
        del colecoes_campos_controlados


        # Captar quantidade de iten do acervo 
        try:
            matadata = requests.get(url_matadados)
            matadata_json = matadata.json()
            tamanho_acervo =  matadata_json['total_items']['publish']
            tamanho_acervo = int(tamanho_acervo)
            qtde_paginas = tamanho_acervo / 96
            qtde_paginas = math.ceil(qtde_paginas)
            print(tamanho_acervo, "itens. ", qtde_paginas, "páginas.")
        except:
            tamanho_acervo = -1
            qtde_paginas = -1



        #print("Iniciou a raspagem do: " + str(nome_unico) + " (" + collection_id + ")")

        check = True
        contador = 1
        error = 0

        while check == True:
            url_atual = url_raspagem + str(contador)
            #print(url_atual)

            if contador == 1:
                try:
                    df_base = pd.read_csv(url_atual)
                except:
                    acervos_que_falharam.append(url)              
                    error = 1
                    check = False
                    break


            else:
                try:
                    df_temp = pd.read_csv(url_atual)
                    df_base = df_base.append(df_temp)
                    #print("Pg: " + str(contador), " com total de " + str(df_base.shape[0]) + " registros.")

                except:
                    check = False
                    break

            contador = contador + 1
            sleep(5)

        if error == 0:
            caminho = "./Acervos dos museus/"+ str(nome_unico) + ".csv"

            df_base["ACERVO"] = nome_unico
            df_base["URL"] = url_raspagem
            df_base["TAINACAN_COLECTION_ID"] = collection_id

            tamanho_baixado = int(df_base.shape[0])

            if tamanho_baixado == tamanho_acervo:
                print("OK - A quantidade de registros do site e baixados são iguais.")
            else:
                print("A quantidade de registros do site e baixados são diferentes.")
                print("Baixados:", df_base.shape[0])
                print("Existentes:", tamanho_acervo)

            df_base.to_csv(caminho)
            total_registros = total_registros + df_base.shape[0]
            print(nome_unico, "raspado com", df_base.shape[0], "registros e com", contador-1, "páginas.")
        else:
            print(nome_unico, "falhou na abordagem principal.")
    except:
        print(nome_unico, "falhou!")
        pass
base_campos_controlados.to_excel("./Campos_com_vocabularios_controlados.xlsx")


 https://museudodiamante.acervos.museus.gov.br/acervo-museologico
titulo-3
numero-de-registro-3
title-3
classe-3
classificacao
tnc_tax_42
local-de-producao
local-producao
tnc_tax_18430
data-de-producao
data-producao
tnc_tax_18432
acervo-3
procedencia-3
autor
autor
tnc_tax_18298
material-tecnica-3
material-tecnica
tnc_tax_11646
termos-de-indexacao-2
altura-cm-2
largura-cm
situacao-2
comprimento-cm
profundidadecm
diametro-cm
circunferenciacm
espessuracm
peso
description-3
data-da-foto-2
marcas-inscricoes-legendas-3
dados-historicos-2
referencias-bibliograficas-arquivisticas-3
condicoes-de-reproducao-2
midias-relacionadas
895 itens.  10 páginas.

 https://museudoouro.acervos.museus.gov.br/acervo/
numero-de-registro-3
title-4
titulo
classificacao-3
classificacao
tnc_tax_40
local-de-producao-2
local-producao
tnc_tax_1865
data-de-producao
data-producao
tnc_tax_1869
autor
autor
tnc_tax_1873
conjunto-com-numero
material-tecnica-3
material-tecnica
tnc_tax_1875
palavras-chave-2
fotografo-3
data

77 itens.  1 páginas.

 https://mhn.acervos.museus.gov.br/moedas-de-ouro/
description-24
title-24
outros-titulos-8
numero-de-registro
classe-2
classificacao
tnc_tax_288
especie-7
pais-3
local-de-producao-13
local-producao
tnc_tax_286366
data-de-producao-13
data-producao
tnc_tax_286372
governante-6
material
material-tecnica
tnc_tax_85144
tecnica-11
estado-de-conservacao
estado-conservacao
tnc_tax_383
diametro-mm-8
peso-g-13
exposicao-3
referencias
exibicoes-2
modo-de-aquisicao
condicoes-de-reproducao-2
1215 itens.  13 páginas.

 https://museucasadahera.acervos.museus.gov.br/indumentaria/
no-de-tombo
numero-tainacan
revisar-numero-tainacan-2
title-2
titulo
autor-fabricante
autor
tnc_tax_55
description-2
materiais
material-tecnica
tnc_tax_39896
tecnica
classificacao
classificacao
tnc_tax_33
epoca-2
data-de-producao-2
data-producao
tnc_tax_90499
altura-frente
altura-costas
circunferencia-cintura
circunferencia-barra
circunferencia-punho-direito-e-esquerdo
circunferencia-punho-esquerdo
comp

In [8]:
len(acervos_que_falharam)

2

In [57]:
list_files = glob("./Acervos dos museus/*.parquet")

if str(len(list_files)) == str(len(url_museus)):
    print("OK - Raspagens realizadas com sucesso, todos os acervos foram baixados.")
else:
    print("Algo parece ter dado errado. Há " + str(len(list_files)) + " arquivos CSV no diretório e " + str(len(url_museus)) + " acervos listados.")

OK - Raspagens realizadas com sucesso, todos os acervos foram baixados.


In [63]:
list_files = glob("./Acervos dos museus/*.csv")

headers = []
counter = 0

print("Processando arquivos...")

for file in list_files:
    counter = counter + 1
    
    temp = pd.read_csv(file, index_col=0)
    temp_headers = temp.columns.values.tolist()
    headers.append(temp_headers)
    
    temp_headers = []
    print(file)
    display(temp.head(1))
    print(temp.shape)
    temp = []
print("\n",counter, " acervos processados.")

Processando arquivos...
./Acervos dos museus\mhn_-_moedas-de-ouro.csv


,Resumo descritivo,Denominação,Número de registro,Classificação,Título,Local de produção,Data de produção,Material/Técnica,Estado de conservação,Dimensões - diâmetro,Dimensões - peso,ACERVO,URL,TAINACAN_COLECTION_ID
0,"Anverso: Rei D. Sancho I a cavalo à direita, s...",Moeda,182344,03 Objetos Pecuniários,Morabitino,(?),1185-1211,Ouro,Bom,"28,6","3,68",mhn_-_moedas-de-ouro,https://mhn.acervos.museus.gov.br/wp-json/tain...,286327


(1215, 14)
./Acervos dos museus\mhn_-_reserva-tecnica.csv


,Número de registro,Outros números,Denominação,Resumo descritivo,Título,Autor,Material/Técnica,Local de produção,Data de produção,Classificação,Estado de conservação,Dimensões - altura,Dimensões - largura,Dimensões - profundidade/comprimento,Dimensões - diâmetro,Dimensões - peso,Condições de reprodução,ACERVO,URL,TAINACAN_COLECTION_ID
0,6199,NaN,Pintura histórica,Cena retratando a chegada da Fragata Constitui...,Chegada da Fragata Constituição ao Rio de Janeiro,Edoardo De Martino,óleo | tela,Brasil,1872,02 Artes Visuais/Cinematográfica > 02.6 Pintura,Bom,"257,00","490,00","0,00",NaN,"0,00","Domínio público, ver http://mhn.acervos.museus...",mhn_-_reserva-tecnica,https://mhn.acervos.museus.gov.br/wp-json/tain...,24


(847, 20)
./Acervos dos museus\museucasabenjaminconstant_-_acervo-museologico.csv


...

(1750, 14)
./Acervos dos museus\museuvictormeirelles_-_mvm-acervo.csv


,Classificação,Denominação,Título,Autor,Local de produção,Data de produção,Material/Técnica,Dimensões,Estado de conservação,Resumo descritivo,Mídias relacionadas,Condições de reprodução,ACERVO,URL,TAINACAN_COLECTION_ID
0,02 Artes Visuais/Cinematográfica > 02.2 Desenho,Desenho,Vista de Ronciglione,Victor Meirelles de Lima,Itália,1853/1856 (circa),Grafite sobre papel,"Obra: 28,7 x 44,0 cm Passe-partout: *Obra enca...",Bom,Agrupamento de edificações sobre um morrote. N...,NaN,NaN,museuvictormeirelles_-_mvm-acervo,https://museuvictormeirelles.acervos.museus.go...,18004


(237, 15)
./Acervos dos museus\museuvillalobos_-_fotografias.csv


,Número de registro,Denominação,Título,Autor,Classificação,Resumo descritivo,Dimensões,Material/Técnica,Data de produção,Local de produção,Estado de conservação,Condições de reprodução,ACERVO,URL,TAINACAN_COLECTION_ID
0,1977.16.108,Fotografia,Villa-Lobos e Arminda,Não identificado,10 Comunicação > 10.1 Documento,Com a dedicatória: “À nossa muito querida e sa...,NaN,Fotografia em preto e branco; papel,18/04/1952,França > Paris,Bom,NaN,museuvillalobos_-_fotografias,https://museuvillalobos.acervos.museus.gov.br/...,1570


(1812, 15)

 23  acervos processados.


In [64]:
headers_tratado = list(itertools.chain(*headers))
counter=collections.Counter(headers_tratado)
print("Contagem dos cabeçalhos apresentados:\n")
print(counter)

Contagem dos cabeçalhos apresentados:

Counter({'Material/Técnica': 23, 'ACERVO': 23, 'URL': 23, 'TAINACAN_COLECTION_ID': 23, 'Classificação': 22, 'Autor': 22, 'Resumo descritivo': 21, 'Local de produção': 21, 'Data de produção': 21, 'Denominação': 20, 'Título': 20, 'Número de registro': 19, 'Dimensões - altura': 15, 'Dimensões - largura': 15, 'Dimensões - profundidade/comprimento': 14, 'Dimensões - diâmetro': 13, 'Dimensões - peso': 13, 'Condições de reprodução': 13, 'Outros números': 7, 'Dimensões - espessura': 7, 'Estado de conservação': 6, 'Situação': 5, 'Dimensões': 5, 'Mídias relacionadas': 4, 'Número': 1, 'Coleção': 1, 'Procedência': 1, 'Data': 1, 'Ano': 1, 'Objeto': 1, 'Descrição': 1})


In [65]:
print('Realizando dataprofiling das bases...\n')
for file in list_files:
    tempo_inicio = time.time()
    
    nome = file.split("\\")[1]
    nome = nome.split(".")[0]
    
    print(nome, "iniciado")
    dados = pd.read_csv(file, low_memory=False, index_col=0)

    # Iniciar data profilling
    titulo = "Data profiling" + nome
    profile = ProfileReport(dados, title=titulo, explorative=True)
    
    arquivo = "./Acervos dos museus/reports/" + nome + ".html"
    # Gerar arquivo html
    profile.to_file(arquivo)

    tempo_fim = time.time()
    tempo_gasto = ((tempo_fim - tempo_inicio)/60)
    tempo_gasto = round(tempo_gasto, 2)
    
    print(nome, "concluído em ", str(tempo_gasto),"segundos.\n")

Realizando dataprofiling das bases...

mhn_-_moedas-de-ouro iniciado


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

...

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

museuvillalobos_-_fotografias concluído em  0.16 segundos.



In [66]:
print("concatenando todas as bases...")
base = pd.read_csv(list_files[0], index_col=0)
list_files.pop(0)
      
for file in list_files:
    nome = file.split("\\")[1]
    nome = nome.split(".")[0]
      
    temp = pd.read_csv(file, index_col=0)
    base = pd.concat([base,temp], axis=0, ignore_index=True)
    print("OK - ", nome)

base.to_excel("./Acervos dos museus/Acervo_IBRAM.xlsx")
profile = ProfileReport(base, title="Data profiling Acervo_IBRAM", explorative=True)

profile.to_file("./Acervos dos museus/reports/Acervo_IBRAM.html")
fim_geral = time.time()
tempo_gasto_geral = ((fim_geral - inicio_geral)/60)
tempo_gasto_geral = round(tempo_gasto_geral, 2)
    
print("Processamento completo em em ", str(tempo_gasto_geral),"segundos.\n")

concatenando todas as bases...
OK -  mhn_-_reserva-tecnica
OK -  museucasabenjaminconstant_-_acervo-museologico
OK -  museucasadahera_-_acervo-museologico
OK -  museucasadahera_-_indumentaria
OK -  museucasahistoricadealcantara_-_objetos
OK -  museudaabolicao_-_acervo_museologico
OK -  museudainconfidencia_-_acervo-museologico
OK -  museudasmissoes_-_acervo-museologico
OK -  museudearqueologiadeitaipu_-_museu-itaipu
OK -  museudeartereligiosaetradicional_-_acervo-museologico
OK -  museudodiamante_-_acervo-museologico
OK -  museudoouro_-_acervo
OK -  museuregionalcasadosottoni_-_acervo-museologico
OK -  museuregionaldecaete_-_acervo
OK -  museuregionaldesaojoaodelrei_-_acervo_museologico
OK -  museusibramgoias_-_museu-casa-da-boa-morte
OK -  museusibramgoias_-_museu-casa-da-princesa
OK -  museusibramgoias_-_museu-das-bandeiras
OK -  museusolarmonjardim_-_acervo
OK -  museusolarmonjardim_-_documentos-historicos
OK -  museuvictormeirelles_-_mvm-acervo
OK -  museuvillalobos_-_fotografias


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\lib\site-packages\pandas_profiling\model\correlations.py:55: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Processamento completo em em  264.87 segundos.

